<a href="https://www.kaggle.com/code/kidkoala/red-wine-quality-prediction?scriptVersionId=183494482" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
#@title Libraries

# Uploading data from Kaggle
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Data handling
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# Visualizations
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots as mp

# Train/Test splitting
from sklearn.model_selection import train_test_split

# Models evaluation metrics
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import roc_auc_score, roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn import tree
import time

# Hyperparameters tuning
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Models
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier

# Get multiple outputs from one cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# Ignore the warning messages
import warnings
warnings.filterwarnings('ignore')

/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv


# Red wine quality prediction

# Objectives

Here I play with [the dataset](https://archive.ics.uci.edu/dataset/186/wine+quality) by P. Cortez, A. Cerdeira et al., that contains data on chemical composition of the Portuguese red wine 'Vinho Verde'. My final goal is to build a model that will predict the quality of the wine from its chemical composition and classify the quality of the wine as good and bad, based on the input.

#  Outline

1. Upload and describe the dataset
2. Explore the data
3. The outliers and skewness
4. Feature engineering
5. Modeling
6. Comparing the models
7. Further recommendations

## Upload and describe the dataset

1. Upload the dataset
2. Count the number of rows and columns
3. Check the data types
4. Look at the data itself
5. Retrieve some descriptive statistics

Let's upload and look at our dataset.

The dataset consists of 1599 rows and 12 columns. We can see the structure of the dataset with its column names and data types. The data includes `fixed acidity`, `volatile acidity`, `citric acid`, `residual sugar`, `chlorides`, `free sulfur dioxide`, `total sulfur dioxide`, `density`,`pH`, `sulphates`, `alcohol`, `quality`. Most of the data are floating numbers, and only the `quality`is categorical, represented by integer number.

From the statistical description, we can retrieve some valuable insights:

* There's no empty data in the dataset - all cells have values;
* The quality rated from 3 to 8;
* There is rather big difference between `total sulfur dioxide's` maximum value and its median - possibly has outliers or skewness;
* The same can be said about `residual sugar` and `free sulfur dioxide`; 
* The data in the dataset mostly normally distributed as we see low deviations between means and medians in the columns;
* There's no column that we could use as an ID of instance. However, we don't need to operate with the unique instances in this particular case;

In [2]:
#@title Description code

# Upload the document

df = pd.read_csv('/kaggle/input/red-wine-quality-cortez-et-al-2009/winequality-red.csv')

# Describing the dataset

print('--------------')
print('Rows, Columns:')
print('--------------')
df.shape
print('---------------')
print("Columns' names:")
print('---------------')
df.columns
print('-----------------')
print('Column / Datatype:')
print('-----------------')
df.dtypes
print('-----------------')
df.head(5)
print('---------------------')
print('Descriptive statistic:')
print('---------------------')
round(df.describe())
print('-----------------------')
print('The sum of null values:')
print('-----------------------')
print(df.isnull().sum())

--------------
Rows, Columns:
--------------


(1599, 12)

---------------
Columns' names:
---------------


Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

-----------------
Column / Datatype:
-----------------


...

-----------------


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


---------------------
Descriptive statistic:
---------------------


,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
count,1599.0,1599.0,1599.0,1599.0,1599.0,1599.0,1599.0,1599.0,1599.0,1599.0,1599.0,1599.0
mean,8.0,1.0,0.0,3.0,0.0,16.0,46.0,1.0,3.0,1.0,10.0,6.0
std,2.0,0.0,0.0,1.0,0.0,10.0,33.0,0.0,0.0,0.0,1.0,1.0
min,5.0,0.0,0.0,1.0,0.0,1.0,6.0,1.0,3.0,0.0,8.0,3.0
25%,7.0,0.0,0.0,2.0,0.0,7.0,22.0,1.0,3.0,1.0,10.0,5.0
50%,8.0,1.0,0.0,2.0,0.0,14.0,38.0,1.0,3.0,1.0,10.0,6.0
75%,9.0,1.0,0.0,3.0,0.0,21.0,62.0,1.0,3.0,1.0,11.0,6.0
max,16.0,2.0,1.0,16.0,1.0,72.0,289.0,1.0,4.0,2.0,15.0,8.0


-----------------------
The sum of null values:
-----------------------
fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64


# Explore the Data

First of all, lets delve into the features the dataset contains. Since the dataset was provided without feature description, we have to investigate those ourselves. Next we will investigate the features and build plots to find correlations with the quality.

In [3]:
def plot_feature_quality(df, feature_name, color_scale):
    grouped_class = df.groupby('quality')[feature_name].mean().reset_index()

    # Create the bar plot
    fig = go.Figure(data=go.Bar(
        x=grouped_class['quality'],
        y=grouped_class[feature_name],
        text=grouped_class[feature_name].apply(lambda x: f'{x:.2f}'),
        marker=dict(
            color=grouped_class[feature_name],
            colorscale=color_scale,
            colorbar=dict(
                title=f'The {feature_name}'
            )
        )
    ))

    fig.update_layout(
        yaxis_title=f'Mean {feature_name}',
        xaxis_title='Quality',
        title={
            'text': f'The mean {feature_name} by wine quality',
            'x': 0.5,
            'y': 0.9,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(
                size=15
            )
        }
    )
    
    return fig

## Acidity

* Fixed acidity doesn't show notable correlations with the quality;
* Volatile acidity negatively correlated with quality;
* PH slightly correlated with quality; 
* We can build synthetic feature `acidity level` based on the pH levels.

Traditionally, total acidity of the wine is divided into `fixed acidity` and `volatile acidity`. Enologist Andrew Waterhouse noted in [his blog post](https://waterhouse.ucdavis.edu/whats-in-wine/fixed-acidity#:~:text=The%20predominant%20fixed%20acids%20found,2%2C000%20mg%2FL%20succinic%20acid.) that one of the major constituents of wine is **fixed acidity**. It impacts the taste, contributing to the sourness and the tartness. It also influences pH, color, and stability to oxidation of wine.

***Fixed acidity*** mostly comes to wine from the grape and gives a wine sourness. It is generally regarded as a positive attribute in terms of taste. Wines from cold climates tend to be more sour as they contain more fixed acids. Winemakers can reduce the sourness of the wine by lowering acidity using various techniques, such as appassimento.

The chart below illustrates that high-quality wines tend to have slightly higher `fixed acidity`.

In [4]:
color_scale = [
    [0.0, '#722255'],
    [1.0, '#e84aae']
]
fig = plot_feature_quality(df, 'fixed acidity', color_scale)
fig.show()

***Volatile Acidity*** refers to steam-distillable acids present in wine due to the winemaking process, primarily acetic acid. Generally, volatile acids are considered detrimental to the taste of wine as they are associated with vinegar flavors, and most winemakers attempt to reduce `volatile acids`.

The chart shows a decrease in `volatile acidity` as the quality of wine rises. Therefore, we can suppose there is a correlation between these two features.

In [5]:
color_scale = [
    [0.0, '#e84aae'],
    [1.0, '#722255']
]

fig = plot_feature_quality(df, 'volatile acidity', color_scale)
fig.show()

Another marker of acidity is ***pH***. The normal acidity range for red wine is considered to be 3-4 pH. The higher the pH level, the softer and 'rounder' the taste becomes. Conversely, the lower the number, the more sour and 'crisp' the wine is. pH [also affects](https://sensorex.com/ph-wine-making/) the color, density, oxidation, protein stability, and bacterial growth and fermentation. Typically, for red wine, the pH range is between 3.0-3.4 (low pH) and 3.6-4.0 (high pH).

pH does not significantly deviate from the quality of samples. However, this feature can affect it non-linearly.

In [6]:
color_scale = [
    [0.0, '#e84aae'],
    [1.0, '#722255']
]

fig = plot_feature_quality(df, 'pH', color_scale)
fig.show()

## Sulfur Dioxide

* There's no notable correlation of Sulfur dioxide with quality;
* We can calculate the amount of `active SO2` based on total SO2 and free SO2.

***Sulfur dioxide (SO2)*** serves as an antioxidant and antimicrobial preservative. It exhibits a negative correlation with pH, meaning that the more acidic your wine is, [the less SO2 you need](https://www.awri.com.au/industry_support/winemaking_resources/frequently_asked_questions/acidity_and_ph/). SO2 also [affects](https://ecsa-chemicals.ch/en/sulphur-dioxide-in-winemaking/) the wine's taste by eliminating the 'stale taste,' enhancing fruity aromas, and reducing rotten and moldy flavors.

However, in the dataset, the total SO2 doesn't show a strong correlation with the quality. Both low and high-quality wines have roughly the same mean amount of total SO2. There's a noticeable spike in the cohort of wines with a 5-point quality, possibly influenced by outliers.

In [7]:
color_scale = [
    [0.0, '#911cff'],
    [1.0, '#420a72']
]

fig = plot_feature_quality(df, 'total sulfur dioxide', color_scale)
fig.show()

***Free Sulfur Dioxide*** is a component of the total SO2 fraction that remains after the chemical reaction of Total SO2 with sugars and ethanol. We can also calculate the amount of active SO2, which is the portion of SO2 that has reacted with the wine.

Low-quality wines in the dataset exhibit a low amount of free SO2. However, there is a noticeable increase at a quality rating of 5, followed by a decline with the rise in the quality of samples. This observed trend may suggest a correlation, potentially influenced by the outliers.

In [8]:
color_scale = [
    [0.0, '#911cff'],
    [1.0, '#420a72']
]

fig = plot_feature_quality(df, 'free sulfur dioxide', color_scale)
fig.show()

## Residual Sugar

* No evidence of correlation between residual sugar and quality;
* We can distinguish `sweetness` as a distinct feature based on the amount of sugar.

***Residual sugar*** is another term for sweetness. It indicates how much sugar remains in the wine after the wine-making process. In the EU, there are rules that regulate how the wine will be labeled based on the amount of residual sugar. Wines with up to 4 g/l are labeled as *dry*, 4-12 g/l as *medium dry*, 12-45 g/l as *medium sweet*, and more than 45 g/l as *sweet*.

Residual sugar doesn't seem to significantly affect the quality of wines in the dataset. However, it may have non-linear correlations, as sweetness is an essential part of the overall taste.

In [9]:
color_scale = [
    [0.0, '#FDFD96'],
    [1.0, '#FFBE6A']
]

fig = plot_feature_quality(df, 'residual sugar', color_scale)
fig.show()

## Chlorides and Sulphates

* There's a notable correlation between chlorides and quality, and sulphates and quality;

***Chlorides and sulphates*** are minerals that contribute to the saltiness of wine. The quantity of these minerals depends mostly on the soil of the specific region where the grapes are grown.

Chlorides tend to decrease as the quality of the wine rises. Conversely, we observe the opposite trend with sulphates, showing a strong positive correlation with quality.

In [10]:
color_scale = [
    [0.0, '#BFBBED'],
    [1.0, '#5A4FCF']
]

fig = plot_feature_quality(df, 'chlorides', color_scale)
fig.show()

color_scale = [
    [0.0, '#DB8780'],
    [1.0, '#A3372E']
]

fig = plot_feature_quality(df, 'sulphates', color_scale)
fig.show()

## Alcohol

* There's a notable correlation between alcohol and quality;  

It is widely common knowledge that alcohol contributes to the 'fullness' and 'boldness' of wine, both of which are appreciated in wine-making. However, a delicate balance is crucial, as some wines could be labeled as 'alcoholic,' indicating an imbalance with an overpowering alcohol taste ([source](https://en.wikipedia.org/wiki/Wine_tasting_descriptors)).

The chart below illustrates a positive correlation between alcohol content and wine quality in the dataset. As the quality score increases with the alcohol content. However, this trend is not as pronounced in poor-quality wines.

In [11]:
color_scale = [
    [0.0, '#5FD85F'],
    [1.0, '#228B22']
]

fig = plot_feature_quality(df, 'alcohol', color_scale)
fig.show()

## Cross-features correlations

Now, let's take a look on the cross-feature correlations.

* alcohol, sulphates and citric acid show notable positive correlations with the wine quality;
* volatile acidity, total sulfur dioxide and density are slightly negatively correlated with quality.
* residual sugar, pH and free sulfur dioxide are not correlated with quality;
* We also observe strong correlations between:
    - density and fixed acidity;
    - chlorides and sulphates;
    - density and alcohol;
    - pH and acidity;
    - residual sugar and density.

In [12]:
#@title Correlation matrix

corr_df = df[['quality', 'fixed acidity', 'volatile acidity',
              'citric acid', 'residual sugar', 'chlorides',
              'free sulfur dioxide', 'total sulfur dioxide',
              'density', 'pH', 'sulphates', 'alcohol']]

correlation_matrix = corr_df.corr()

# Create the heatmap

heatmap = go.Heatmap(
    z=correlation_matrix.values,
    x=correlation_matrix.columns,
    y=correlation_matrix.index,
    colorscale='agsunset'
)

layout = go.Layout(
    title={
        'text': 'Cross-features correlations',
        'x': 0.5,
        'y': 0.9,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 15}
    },
    xaxis=dict(title='Columns'),
    yaxis=dict(title='Columns'),
)

annotations = []

for i, row in enumerate(correlation_matrix.values):
    for j, value in enumerate(row):
        annotations.append(
            dict(
                x=correlation_matrix.columns[j],
                y=correlation_matrix.index[i],
                text=f'{value:.2f}',
                showarrow=False,
                font=dict(color='white'),
            )
        )

layout['annotations'] = annotations

fig = go.Figure(data=heatmap, layout=layout)

fig.show()

# The Outliers and Skewness

1. Check for the outliers
2. Explore the distributions of the features
3. Normalize data

As we have already seen, there's no missing values in the dataset. It ease the job, as we don't need to handle those blanks. However, there could be some outliers that can affect the modeling. Let's find those out.

As the following charts illustrates, there're numbers of outliers in most of the columns. Let's check how skewed the dataset is.

In [13]:
#@title Box plots

fig = mp(rows=4, cols=3,
         subplot_titles=('F.Acidity', 'V.Acidity', 'C.Acidity',
                         'Residual sugar', 'Chlorides',
                         'Free sulfur dioxide', 'total sulfur dioxide',
                         'density', 'pH', 'sulphates'),
         specs=
          [
              [{}, {}, {}],
              [{}, {}, {}],
              [{}, {}, {}],
              [{}, {}, {}]
              ]
         )

fig = fig.add_traces([
                go.Box(y=df['fixed acidity'],
                       name = 'F.Acidity'),
                go.Box(y=df['volatile acidity'],
                       name = 'V.Acidity'),
                go.Box(y=df['citric acid'],
                       name = 'C.Acidity'),
                go.Box(y=df['residual sugar'],
                       name = 'Residual sugar'),
                go.Box(y=df['chlorides'],
                       name = 'Chlorides'),
                go.Box(y=df['free sulfur dioxide'],
                       name = 'Free sulfur dioxide'),
                go.Box(y=df['total sulfur dioxide'],
                       name = 'Total sulfur dioxide'),
                go.Box(y=df['density'],
                       name = 'Density'),
                go.Box(y=df['pH'],
                       name = 'pH'),
                go.Box(y=df['sulphates'],
                       name = 'Sulphates')],
               rows=[1, 1, 1, 2, 2, 2, 3, 3, 3, 4],
               cols=[1, 2, 3, 1, 2, 3, 1, 2, 3, 1])

fig.update_layout(
        title={
        'text': "Features' Distributions",
        'x': 0.5,
        'y': 0.975,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 20}
    },
        height=1350)

# Checking for skewness
print('--------------------')
print('Skewness of the data:')
print('--------------------')
round(df.skew(),2)

--------------------
Skewness of the data:
--------------------


fixed acidity           0.98
volatile acidity        0.67
citric acid             0.32
residual sugar          4.54
chlorides               5.68
free sulfur dioxide     1.25
total sulfur dioxide    1.52
density                 0.07
pH                      0.19
sulphates               2.43
alcohol                 0.86
quality                 0.22
dtype: float64

As we can see, the most of the data are right-side skewed. The most effective ways to ways to address the skewness are logarithmic transformations and square root transformation. We will apply the square root transformation for the most of the values. However, the dataset demonstrates sufficient skewness in `residual sugar`, `chlorides` and `sulphates`. Let's investigate those features closely and then decide what to do with them.

In [14]:
#@title Square rooting

df['fixed acidity'] = np.sqrt(df['fixed acidity'])
df['volatile acidity'] = np.sqrt(df['volatile acidity'])
df['free sulfur dioxide'] = np.sqrt(df['free sulfur dioxide'])
df['total sulfur dioxide'] = np.sqrt(df['total sulfur dioxide'])
df['alcohol'] = np.sqrt(df['alcohol'])

## Residual sugar

There is a big dispersion of residual sugar in all the quality groups. Usually, considering such a distribution and the low impact on the dependent variable, I would drop the whole feature from the dataset. But this time, I want to save the `residual sugar` as I see sweetness is essential part of the taste.

Earlier I mentioned, that we can group the wine according to its sugar amount. So further, I will transform `residual sugar` value into `sweetness` feature. I also will drop the column `residual sugar` then.

In [15]:
#@title Sugar

fig = px.histogram(df['residual sugar'],
                   x="residual sugar",
                   y="residual sugar",
                   color=df['quality'],
                   marginal="box")

fig.update_layout(
    yaxis_title='Sum of Residual Sugar',
    xaxis_title='Residual Sugar',
    title={
        'text': 'The Distribution of Residual Sugar by Quality',
        'x': 0.5,
        'y': 0.95,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(
            size=15
        )
    }
)


## Chlorides and Sulphates

Chlorides and Sulphates demonstrate high dispersions in quality groups 5 and 6. It can affect the overall output of the models. The possible solutions could be:
- To drop the outliers group-based;
- To apply different type of transformation, i.e. `log()` or `log10()` transformations.
- Do nothing and see how those outliers will affect our model.

As the outliers are essential part of the features that influence the dependent variable, I definitely won't drop them. It seems like application of the `log10` transformation is appropriate in this case. Log10 is more powerful than simple logarithmic transformation, and we see some major skewness in the features.

In [16]:
#@title Salts

fig = px.histogram(df['chlorides'],
                   x="chlorides",
                   y="chlorides",
                   color=df['quality'],
                   marginal="box")

fig.update_layout(
    yaxis_title='Sum of Chlorides',
    xaxis_title='Chlorides',
    title={
        'text': 'The Distribution of Chlorides by Quality',
        'x': 0.5,
        'y': 0.95,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(
            size=15
        )
    }
)

fig = px.histogram(df['sulphates'],
                   x="sulphates",
                   y="sulphates",
                   color=df['quality'], marginal="box")

fig.update_layout(
    yaxis_title='Sum of Sulphates',
    xaxis_title='Sulphates',
    title={
        'text': 'The Distribution of Sulphates by Quality',
        'x': 0.5,
        'y': 0.95,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(
            size=15
        )
    }
)

We still have some skewness in `chlorides` and `sulphates`, but now we can proceed with that. Also, as I mentioned earlier, we will transform the `residual sugar` to `sweetness` feature, then drop it.

In [17]:
#@title Reducing skewness of salts

df['chlorides'] = np.log10(df['chlorides'])
df['sulphates'] = np.log10(df['sulphates'])

print('---------------------')
print('Skewness of the data:')
print('---------------------')
round(df.skew(),2)

---------------------
Skewness of the data:
---------------------


fixed acidity           0.69
volatile acidity        0.11
citric acid             0.32
residual sugar          4.54
chlorides               1.75
free sulfur dioxide     0.48
total sulfur dioxide    0.64
density                 0.07
pH                      0.19
sulphates               0.92
alcohol                 0.76
quality                 0.22
dtype: float64

# Feature engineering

1. Creating new features
    - active SO2
    - sweetness of wine
    - low pH, high pH
2. Encoding quality good and bad 1/0
3. Drop sugar and quality
4. Scaling the data

First of all, lets create some new features. As I investigated in the first part, I can group the sweetness of wine based on its residual sugar. I also can calculate the amount of active SO2 that interacted with alcohol and sugar in the wine-making process. And then I can group wines based on the pH acidity.

As my goal is to build a model that can classify the wines as good and bad, I should encode the quality into the binary scale.

After I make those manipulations, I will drop the `residual sugar` column as it has abnormal distribution and low correlations with quality. I will also drop `quality` as we will work with encoded value from now.

In [18]:
#@title New features

# Creating active SO2 as a feature

df['active SO2'] = df['total sulfur dioxide'] - df['free sulfur dioxide']

# Creating sweetness as a value

conditions = [
    (df['residual sugar'] <= 4),
    (df['residual sugar'] > 4) & (df['residual sugar'] <= 12),
    (df['residual sugar'] > 12) & (df['residual sugar'] <= 45),
    (df['residual sugar'] > 45)
]

choices = [1, 2, 3, 4] # 1-dry, 2-semi dry, 3-medium sweet, 4-sweet

df['sweetness'] = np.select(conditions, choices, default=np.nan)
df['sweetness'] = df['sweetness'].astype(int)

# Add low and high pH as a feature

conditions = [
    (df['pH'] <= 3.5),
    (df['pH'] >= 3.5) & ((df['pH'] < 3.6)),
    (df['pH'] >= 3.6)
]

choices = [1, 2, 3] # 1 - high, 2 - medium, 3 - low

df['pH acidity'] = np.select(conditions, choices, default=np.nan)
df['pH acidity'] = df['pH acidity'].astype(int)

# Add quality encoding

conditions = [
    (df['quality'] <= 5),
    (df['quality'] > 5)
]

choices = [0, 1] # 0 - low quality, 1 - high quality

df['quality encoding'] = np.select(conditions, choices, default=np.nan)
df['quality encoding'] = df['quality encoding'].astype(int)

# Trim the unnecessary features

df.drop('residual sugar', axis = 1, inplace=True)
df.drop('quality', axis = 1, inplace=True)

Just before we start, let's take a look on how those quality encoding has split our dataset.

As we can see, the wines are almost equally distributed between good and bad. We don't need to adjust this values and can begin with the modeling part.

In [19]:
#@title Good / Bad

fig = px.pie(df, names='quality encoding',
             title='The Distribution of the Good (1) and The Bad (0) Wines',
             labels={'quality encoding':'Quality'},
             width=700,
             height=500,
             hole=0.3)
fig.show()

In [20]:
#@title Scaling

columns = ['fixed acidity', 'volatile acidity', 'citric acid',
         'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 
         'density', 'pH', 'sulphates', 'alcohol']

scaler = MinMaxScaler()
df[columns] = scaler.fit_transform(df[columns])

# Modeling

1. Split the dataset
2. Automate the hyperparameter tuning
3. Modeling
    * Logistic Regression
    * Decision Tree
    * Random Forest
    * XGBoost
4. Evaluate the results

## Splitting the dataset
First, let's split the dataset to the test and the train datasets. We will set the train/test datasets as 80/20 from the original one.

In [21]:
#@title Split

X = df.drop('quality encoding', axis=1)
y = df['quality encoding']

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    random_state=42)

## Automate the hyperparameter tuning

We will use GridSearch and Random Search algorithms for hyperparameter tuning. GridSearch helps us to find the best combination of hyperparameters for the best model outcomes. There are lots of models for that task. However, GridSearch is rather good when it comes to fitting models with comparatively small number of hyperparameters.

When it comes to models with large or continuous hyperparameter space GridSearch could be not the best options as it require sufficient computational resources. The alternative can be Randomized Search. Randomized Search allows us to define a range for each hyperparameter and randomly samples from these range. It's a good option, when the computational cost is an issue.

So, we will apply Grid Search to simple models, like Logistic Regression and Decision Tree. And we will apply Randomized Search for Random Forest and XGBoost models.

In [22]:
#@title Grid Search

def hyperp_search(classifier, parameters, cv):
    gs = GridSearchCV(classifier,
                      parameters,
                      cv=cv,
                      scoring = 'f1',
                      verbose=0,
                      n_jobs=-1
                      )
    tic = time.perf_counter()
    gs = gs.fit(X_train, y_train)
    toc = time.perf_counter()

    print("f1_train: %f using %s in %.3f seconds" % (gs.best_score_,
                                                     gs.best_params_,
                                                     toc - tic))

    best_model = gs.best_estimator_
    y_pred = best_model.predict(X_test)
    y_pred_train = best_model.predict(X_train)

    # evaluate predictions
    print("           train    test ")
    print("-------------------------")
    print("f1         %.3f    %.3f" % (f1_score(y_train, y_pred_train),
                                       f1_score(y_test, y_pred)))
    print("accuracy   %.3f    %.3f" % (accuracy_score(y_train, y_pred_train),
                                       accuracy_score(y_test, y_pred)))
    print("precision  %.3f    %.3f" % (precision_score(y_train, y_pred_train),
                                       precision_score(y_test, y_pred)))
    print("recall     %.3f    %.3f" % (recall_score(y_train, y_pred_train),
                                       recall_score(y_test, y_pred)))
    print("")


    # Calculate the ROC curve

    fpr, tpr, thresholds = roc_curve(y_test, y_pred)
    roc_auc = auc(fpr, tpr) # Calculate the area under the ROC curve

    # ROC curve plot

    fig = px.area(
        x=fpr, y=tpr,
        title=f'ROC Curve (AUC={auc(fpr, tpr):.4f})',
        labels=dict(x='False Positive Rate', y='True Positive Rate'),
        width=700, height=500
    )
    fig.add_shape(
        type='line', line=dict(dash='dash'),
        x0=0, x1=1, y0=0, y1=1
    )

    fig.show()

    # Confusion matrix plot

    cm = confusion_matrix(y_test, y_pred)

    fig = px.imshow(
        cm,
        labels=dict(x="Predicted Label", y="True Label", color="Count"),
        x=['Predicted 0', 'Predicted 1'],
        y=['Actual 0', 'Actual 1'],
        text_auto=True,
        color_continuous_scale="PuBu",
        title="Confusion Matrix",
        width=700, height=500
    )

    fig.show()

    return gs.best_estimator_

In [23]:
#@title Randomized Search

def hyperp_search_random(classifier, parameters, cv):
    gs = RandomizedSearchCV(classifier,
                            parameters,
                            cv=cv,
                            scoring = 'f1',
                            verbose=0,
                            n_jobs=-1,
                            random_state = 82
                           )
    tic = time.perf_counter()
    gs = gs.fit(X_train, y_train)
    toc = time.perf_counter()

    print("f1_train: %f using %s in %.3f seconds" % (gs.best_score_,
                                                     gs.best_params_,
                                                     toc - tic))

    best_model = gs.best_estimator_
    y_pred = best_model.predict(X_test)
    y_pred_train = best_model.predict(X_train)

    # Evaluate prediction
    print("           train    test ")
    print("-------------------------")
    print("f1         %.3f    %.3f" % (f1_score(y_train, y_pred_train),
                                       f1_score(y_test, y_pred) ))
    print("accuracy   %.3f    %.3f" % (accuracy_score(y_train, y_pred_train),
                                       accuracy_score(y_test, y_pred) ))
    print("precision  %.3f    %.3f" % (precision_score(y_train, y_pred_train),
                                       precision_score(y_test, y_pred) ))
    print("recall     %.3f    %.3f" % (recall_score(y_train, y_pred_train),
                                       recall_score(y_test, y_pred) ))
    print("")

    # Calculate the ROC curve

    fpr, tpr, thresholds = roc_curve(y_test, y_pred)
    roc_auc = auc(fpr, tpr) # Calculate the area under the ROC curve

    # ROC curve plot

    fig = px.area(
        x=fpr, y=tpr,
        title=f'ROC Curve (AUC={auc(fpr, tpr):.4f})',
        labels=dict(x='False Positive Rate', y='True Positive Rate'),
        width=700, height=500
    )
    fig.add_shape(
        type='line', line=dict(dash='dash'),
        x0=0, x1=1, y0=0, y1=1
    )

    fig.show()

    # Confusion matrix plot

    cm = confusion_matrix(y_test, y_pred)

    fig = px.imshow(
        cm,
        labels=dict(x="Predicted Label", y="True Label", color="Count"),
        x=['Predicted 0', 'Predicted 1'],
        y=['Actual 0', 'Actual 1'],
        text_auto=True,
        color_continuous_scale="PuBu",
        title="Confusion Matrix",
        width=700, height=500
    )

    fig.show()

    return gs.best_estimator_

## Logistic regression

Logistic Regression is simple yet powerful model for binary classifications. In most real-life projects we use it for both - having solid prediction, and having a benchmark for more sophisticated models.

After we do modeling, we can retrieve the most valuable features. For the LR model they are: `alcohol`, `sulphates` and `volatile acidity`.

In [24]:
#@title LR

classifier = LogisticRegression()
parameters = {"C":[1e-4,1e-3,1e-2,1,10,1000], "max_iter":[500,1000,5000] }
cv = 5
best_log = hyperp_search(classifier,parameters,cv)

# Finding the best features

importance_log = pd.DataFrame()
importance_log["feature"]=X_train.columns
importance_log["weight"]=best_log.coef_[0].round(2)
importance_log.sort_values(by=['weight'], inplace=True)

# Plotting the features

fig = px.bar(importance_log,
             x='feature',
             y='weight',
             color='feature',
             labels={'feature': 'Feature',
                     'weight': 'Weight'},
             title='Coefficient Analysis for Logistic Regression',
             color_discrete_sequence=px.colors.sequential.Blues_r,
             text='weight')

fig.update_layout(
    xaxis=dict(tickangle=90, categoryorder='total descending'),
    legend=dict(title='Feature')
)

f1_train: 0.758315 using {'C': 1, 'max_iter': 500} in 2.653 seconds
           train    test 
-------------------------
f1         0.769    0.759
accuracy   0.758    0.738
precision  0.777    0.781
recall     0.762    0.737



## Decision Tree

Decision Tree is a hierarchical model for classification tasks. The model provides comparatively high predictive performance and great explainability.

Our DT found the best features in `alcohol`, `sulphates` and `total sulphur dioxide`.

In [25]:
#@title DT

classifier = DecisionTreeClassifier()
parameters = {'criterion': ['entropy','gini'],
              'max_depth': [4,5,6,8,10,12],
              'min_samples_split': [5,10,20],
              'min_samples_leaf': [5,10,20]}
cv = 5
best_tree = hyperp_search(classifier,parameters,cv)

# Finding the best feature

importance_tree = tree.export_text(best_tree,
                                   feature_names=X_test.columns.tolist(),
                                   max_depth=2)

print("-------------------------------")
print("The most valuable features are: ")
print("-------------------------------")
print(importance_tree)

f1_train: 0.752901 using {'criterion': 'gini', 'max_depth': 12, 'min_samples_leaf': 10, 'min_samples_split': 20} in 2.872 seconds
           train    test 
-------------------------
f1         0.847    0.751
accuracy   0.841    0.728
precision  0.863    0.771
recall     0.831    0.732



-------------------------------
The most valuable features are: 
-------------------------------
|--- alcohol <= 0.36
|   |--- sulphates <= 0.31
|   |   |--- alcohol <= 0.23
|   |   |   |--- truncated branch of depth 8
|   |   |--- alcohol >  0.23
|   |   |   |--- truncated branch of depth 6
|   |--- sulphates >  0.31
|   |   |--- total sulfur dioxide <= 0.45
|   |   |   |--- truncated branch of depth 10
|   |   |--- total sulfur dioxide >  0.45
|   |   |   |--- truncated branch of depth 4
|--- alcohol >  0.36
|   |--- sulphates <= 0.32
|   |   |--- alcohol <= 0.50
|   |   |   |--- truncated branch of depth 4
|   |   |--- alcohol >  0.50
|   |   |   |--- truncated branch of depth 3
|   |--- sulphates >  0.32
|   |   |--- total sulfur dioxide <= 0.37
|   |   |   |--- truncated branch of depth 7
|   |   |--- total sulfur dioxide >  0.37
|   |   |   |--- truncated branch of depth 3



## Random Forest

Random Forest is one of the most popular models for classification tasks. It powerful as it combines a bunch of decision trees and sophisticated algorithm of cross-validation under hood.

After modeling, we can visualize what features are the most important for RF to make a prediction. We calculate feature importance based on the mean decrease in impurity (MDI). It computed as the mean and standard deviation of accumulation of the impurity decrease within each tree of the forest. Based on MSI the most valuable features are `alcohol`, `sulphates`, `active sulphur dioxide` and `volatile acidity`.

In [26]:
#@title RF

classifier = RandomForestClassifier()
parameters = {'criterion': ['entropy'],
              'n_estimators' : [50,100,500,1000],
              'max_depth': range(2,20,2),
              'min_samples_leaf':[100,250,500]}
cv = 5
best_rf = hyperp_search_random(classifier,parameters,cv)

# Finding the best feature

feature_importances = best_rf.feature_importances_
importance_rf = pd.DataFrame({'Feature': X_train.columns,
                              'Weight': feature_importances}).round(3)
importance_rf = importance_rf.sort_values(by='Weight', ascending=False)

# Plotting the features importance

fig = px.bar(importance_rf, x='Feature', y='Weight', color='Feature',
             title='Feature Importances for RandomForest Classifier',
             color_discrete_sequence=px.colors.sequential.Blues_r,
             text='Weight')

fig.update_layout(xaxis=dict(tickangle=90),
                  legend=dict(title='Feature'))

f1_train: 0.769820 using {'n_estimators': 500, 'min_samples_leaf': 100, 'max_depth': 2, 'criterion': 'entropy'} in 23.454 seconds
           train    test 
-------------------------
f1         0.787    0.767
accuracy   0.768    0.738
precision  0.764    0.762
recall     0.811    0.771



## XGBoost

Extreme gradient boosting is a scalable distributed gradient-boosted model. It works as an ensemble of decision trees. It's a powerful tool for such tasks as regression, classification and ranking.

Afterwards, we can observe that `alcohol`, `sulphates`, `pH acidity` and `volatile acidity` are contributing the most to the model's prediction.

In [27]:
#@title XGB

classifier = XGBClassifier()
parameters = {"learning_rate":[0.001, 0.01, 0.1],
              "min_child_weight" : [5, 10, 20],
              "max_depth":[2,4,6,8,10,12],
              "subsample" : [0.5],
              "colsample_bytree" : [0.5],
              "enable_categorical" : [True],
              "objective" : ['binary:logistic'],
              "n_estimators":[50, 100, 500],
              "seed" :[42]}
cv = 10
best_xgb = hyperp_search_random(classifier,parameters,cv)

# Finding the best feature

feature_importances = best_xgb.feature_importances_
importance_xgb = pd.DataFrame({'Feature': X_train.columns,
                               'Weight': feature_importances}).round(3)
importance_xgb = importance_xgb.sort_values(by='Weight', ascending=False)

# Plotting the features importance

fig = px.bar(importance_xgb, x='Feature', y='Weight', color='Feature',
             title='Feature Importances for XGBoosting',
             color_discrete_sequence=px.colors.sequential.Blues_r,
             text=importance_xgb['Weight'].apply(lambda x: f'{x:.3f}'),
             hover_data=['Feature', 'Weight'])

fig.update_layout(xaxis=dict(tickangle=90), legend=dict(title='Feature'))

f1_train: 0.784651 using {'subsample': 0.5, 'seed': 42, 'objective': 'binary:logistic', 'n_estimators': 100, 'min_child_weight': 10, 'max_depth': 4, 'learning_rate': 0.1, 'enable_categorical': True, 'colsample_bytree': 0.5} in 4.417 seconds
           train    test 
-------------------------
f1         0.851    0.790
accuracy   0.844    0.772
precision  0.857    0.815
recall     0.845    0.765



## Comparing the models

I compare the models based on F1 score and accuracy. F1 score provides a balance between precision and recall outputs of the model. Accuracy is easy to understand as it's a ratio of correctly predicted instances in the dataset.

The XGBoost model has demonstrated the best performance among all. It gave us pretty high level of F1 and accuracy on the test dataset.
The Random Forest and Logistic Regressions are also performed good. However, Random Forest showed slightly higher accuracy and majorly higher F1.
The Decision Tree didn't performed well in this project.

The most valuable feature to predict the quality of wine are the same across the models:

1. alcohol;
2. sulphates;
3. total sulphur dioxide;
4. active sulphur dioxide;
5. volatile acidity;

In [28]:
#@title Comparison

models = ["Logistic Regression",
          'Decision Tree',
          'Random Forest',
          'XGBoost']

f1_scores = [f1_score(y_test, best_log.predict(X_test)),
             f1_score(y_test, best_tree.predict(X_test)),
             f1_score(y_test, best_rf.predict(X_test)),
             f1_score(y_test, best_xgb.predict(X_test))]
accuracy_scores = [accuracy_score(y_test, best_log.predict(X_test)),
                   accuracy_score(y_test, best_tree.predict(X_test)),
                   accuracy_score(y_test, best_rf.predict(X_test)),
                   accuracy_score(y_test, best_xgb.predict(X_test))]

table = pd.DataFrame({"Model": models,
                      "F1 Score": f1_scores,
                      "Accuracy": accuracy_scores})

table = table.sort_values(by="F1 Score", ascending=False)

fig = go.Figure(data=[go.Table(
    header=dict(values=['Model',
                        'F1 Score',
                        'Accuracy']),
    cells=dict(values=[table["Model"],
                       table["F1 Score"].round(3),
                       table["Accuracy"].round(3)])
)])

fig.update_layout(
    title={
        'text': 'Comparing the models',
        'x': 0.5,
        'y': 0.9,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 15}
    },
    width=1000,
    height=400)

In [29]:
#@title ROC comparison

log_fpr, log_tpr, _ = roc_curve(y_test, best_log.predict(X_test))
tree_fpr, tree_tpr, _ = roc_curve(y_test, best_tree.predict(X_test))
rf_fpr, rf_tpr, _ = roc_curve(y_test, best_rf.predict(X_test))
xgb_fpr, xgb_tpr, _ = roc_curve(y_test, best_xgb.predict(X_test))
log_auc = auc(log_fpr, log_tpr)
tree_auc = auc(tree_fpr, tree_tpr)
rf_auc = auc(rf_fpr, rf_tpr)
xgb_auc = auc(xgb_fpr, xgb_tpr)

models = ["Logistic Regression",
          "Decision Tree",
          "Random Forest",
          "XGBoost"]

fprs = [log_fpr, tree_fpr, rf_fpr, xgb_fpr]
tprs = [log_tpr, tree_tpr, rf_tpr, xgb_tpr]
auc_scores = [log_auc, tree_auc, rf_auc, xgb_auc]

fig = go.Figure()

# Add the reference line

fig = fig.add_shape(
        type='line', line=dict(dash='dash'),
        x0=0, x1=1, y0=0, y1=1
)

# Add ROC curves for each model

for i in range(len(models)):
    name = f"{models[i]} (AUC={auc_scores[i]:.2f})"
    fig = fig.add_trace(go.Scatter(x=fprs[i],
                                   y=tprs[i],
                                   name=name,
                                   mode='lines'))

# Update layout

fig.update_layout(
    title={
        'text': 'ROC Curves Across the Models',
        'x': 0.5,
        'y': 0.9,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': {'size': 15}
    },
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    yaxis=dict(scaleanchor="x", scaleratio=1),
    xaxis=dict(constrain='domain'),
    width=700, height=500
)

# Further recommendations

Here I will outline some steps that one can take for the further research.

1. Fine-tune RF and XGBoost models to address slight over-fitting;
2. Enlarge the dataset to provide more instances for training the models;
    - One-hot encoding for the sweetness;
    - Create some synthetic features with crossing;
3. Try different modeling approach;